<a href="https://colab.research.google.com/github/JihoonPark99/NLP_study/blob/main/_4_%EB%AC%B8%EC%84%9C_%EB%B6%84%EB%A5%98(Document_Classification).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 문서 분류(Document Classification)

## 데이터 준비

* 문서 분류에 필요한 데이터는 `scikit-learn`이 제공하는 20개의 주제를 가지는 뉴스그룹 데이터를 사용
* 텍스트는 `CounterVectorizer`를 거쳐 DTM 행렬로 변환
* DTM 행렬은 문서에 등장하는 단어들을 빈도 수 별로 표현한 행렬 'Document Term Matrix'


In [ ]:
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split

news = fetch_20newsgroups()

x = news.data
y = news.target

cv = CountVectorizer()
x = cv.fit_transform(x)

x_train, x_test, y_train, y_test = train_test_split(x,y,test_size=0.3)
print(x_train.shape, x_test.shape, y_train.shape, y_test.shape)

(7919, 130107) (3395, 130107) (7919,) (3395,)


In [ ]:
print(x_train[0])
# (0번째문서, n번째 인덱스를 가지는 값) n'번 등장했다.

  (0, 56979)	1
  (0, 111322)	1
  (0, 114731)	2
  (0, 87620)	1
  (0, 95162)	1
  (0, 64095)	1
  (0, 90379)	1
  (0, 76032)	1
  (0, 28615)	1
  (0, 114455)	1
  (0, 115475)	1
  (0, 66608)	1
  (0, 73201)	1
  (0, 90252)	1
  (0, 128402)	1
  (0, 62221)	1
  (0, 94362)	1
  (0, 88363)	1
  (0, 119451)	1
  (0, 25399)	2
  (0, 75901)	1
  (0, 108799)	1
  (0, 58830)	1
  (0, 51268)	2
  (0, 70010)	1
  :	:
  (0, 60150)	2
  (0, 39415)	3
  (0, 48351)	1
  (0, 26085)	1
  (0, 93582)	1
  (0, 67186)	1
  (0, 27721)	1
  (0, 128084)	1
  (0, 33331)	2
  (0, 92999)	1
  (0, 121265)	1
  (0, 3802)	1
  (0, 56195)	1
  (0, 96852)	1
  (0, 67796)	1
  (0, 68253)	1
  (0, 86694)	1
  (0, 114309)	1
  (0, 67670)	1
  (0, 50695)	2
  (0, 64171)	2
  (0, 96697)	1
  (0, 38213)	3
  (0, 33859)	2
  (0, 95031)	1


## scikit-learn을 이용한 문서 분류

In [ ]:
from sklearn.metrics import accuracy_score

### Logistic Regression

:* Logistic Regression은 특성상 다중 분류에는 적합하지 않음
- 여기서 사용할 순 없음.우린 20개의 클래스가 있기 때문문

In [ ]:
from sklearn.linear_model import LogisticRegression

LR = LogisticRegression()
LR.fit(x_train, y_train)
pred = LR.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.8733431516936672


/usr/local/lib/python3.8/dist-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


### 나이브 베이즈 분류기(Naive Bayes Classification)
- 베이즈 정리를 적용한 확률적 분류 알고리즘
- 모든 특성들이 독립임을 가정(naive 가정)
- 입력 특성에 따라 3개의 분류기 존재
  - 가우시안 나이브 베이즈 분류기
  - 베리누이 나이브 베이즈 분류기
  - 다항 나이브 베이즈 분류기
- https://angeloyeo.github.io/2020/01/09/Bayes_rule.

#### DTM을 이용한 Naive Bayes

In [ ]:
##우리는 다항 나이브베이즈 분류기를 사용할것임
from sklearn.naive_bayes import MultinomialNB

NB = MultinomialNB()
NB.fit(x_train, y_train)
pred = NB.predict(x_test)
acc = accuracy_score(pred,y_test)
print(acc)

0.8309278350515464


#### tf-idf를 이용한 정확도 향상

In [ ]:
from sklearn.feature_extraction.text import TfidfTransformer

##x_train과 x_test를 tf-idf로 바꿔주기기
tfidf = TfidfTransformer()
x_train_tf = tfidf.fit_transform(x_train)
x_test_tf = tfidf.fit_transform(x_test)

##바꾼것으로 학습 하기 
NB.fit(x_train_tf, y_train)
pred = NB.predict(x_test_tf)
acc = accuracy_score(pred, y_test)
print(acc)

0.8394698085419735


### Support Vector Machine

- 회귀,분류,이상치 탐지 등에 사용되는 지도학습 방법
- 클래스 사이의 경계에 위치한 데이터포인트를 서포트 벡터(support vector)라고 함
- 각 서포트 벡터가 클래스 사이의 결정경계를 구분하는데 얼마나 중요한지를 복습
- 각 서포트 벡터 사이의 마진이 가장 큰 방향으로 학습
- 서포트 벡터 까지의 거리와 서포트 벡터의 중요성을 기반으로 예측을 수행행
- https://hleecaster.com/ml-svm-concept/

In [ ]:
###SVM공부
from sklearn.svm import SVC
classifier = SVC(kernel = 'linear')
training_points = [[1, 2], [1, 5], [2, 2], [7, 5], [9, 4], [8, 2]]
labels = [1, 1, 1, 0, 0, 0]
classifier.fit(training_points, labels) 

SVC(kernel='linear')

In [ ]:
##분류됐는지 확인
print(classifier.predict([[3, 2]]))

##서포트벡터 확인
print(classifier.support_vectors_)

[1]
[[7. 5.]
 [8. 2.]
 [2. 2.]]


In [ ]:
##이상치 허용
classifier = SVC(C = 0.01)
#기본값은 1
#당연히 C의 최적 값은 데이터에 따라 다르다. 결국 여러가지 C값을 넣어보면서 모델을 검증하는 수밖에 없다.

In [ ]:
#선형데이터가 아닐땐, kernel을 지정해서 해결하자!!
from sklearn.svm import SVC
classifier = SVC(kernel = 'poly')

In [ ]:
from sklearn import svm

SVM = svm.SVC(kernel='linear')
SVM.fit(x_train, y_train)
pred = SVM.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.8300441826215023


### Decision Tree

- 분류와 회귀에 사용되는 지도 학습 방법
- 데이터 특성으로 부터 추론된 결정 규칙을 통해 값을 예측
- if-then-else 결정규칙을 통해 데이터 학습
- 트리의 깊이가 깊을수록 복잡한 모델

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn import tree
import graphviz
DT = DecisionTreeClassifier()
DT.fit(x_train, y_train)
pred = DT.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)

0.6185567010309279


In [ ]:
# import matplotlib.pyplot as plt
# ## Plot Tree with plot_tree
# fig = plt.figure(figsize=(15, 8))
# DT.plot_tree

### XGBoost

- 트리가반의 앙상블 기법
- 분류에 있어서 다른 알고리즘보다 좋은 예측 성능을 보여줌
- XGBoost는 GBM기반이지만, GBM의 단점인 느린 수행시간과 과적합 규제 부재 등의 문제를 해결
- 병렬CPU환경에서 빠르게 학습 가능


In [ ]:

from xgboost import XGBClassifier

xgb = XGBClassifier(n_estimtors = 30, 
                    learning_rate=0.05,
                    max_depth = 3)
xgb.fit(x_train, y_train)
pred = xgb.predict(x_test)
acc = accuracy_score(pred, y_test)
print(acc)


0.7737849779086893


##교차 검증

* 일반 검증은 학습 데이터가 테스트 데이터로 사용되지 않음
* 교차 검증은 데이터를 n개의 집합으로 나누어 정확도를 계산해 학습 데이터로 사용된 데이터도 테스트 데이터로 사용
* 교차 검증을 사용하면 일반 검증보다 모델의 일반화가 잘 되어 있는지 평가 가능
* 앞서 구성한 나이브 베이즈 모델을 교차 검증

In [ ]:
from sklearn.model_selection import cross_val_score

scores = cross_val_score(NB,x,y,cv=5)
print(scores, scores.mean())

[0.83870968 0.83826779 0.82368537 0.83031374 0.83642794] 0.833480903927519


* 교차 검증을 통해 일반 검증보다 좀 더 일반화된 모델 생성 가능
* 교차 검증은 일반 검증에 비해 n번 검증을 해 비용이 더 많이 소요

## 정밀도와 재현률 



# 코드로 형식 지정됨


* 정밀도(precision)는 양성 클래스(정답)으로 예측한 샘플이 양성 클래스일 확률을 의미
* 모델이 얼마나 양성 클래스를 잘 예측하는지를 나타냄
* 재현률(recall)은 양성 클래스인 샘플에서 모델이 양성 클래스로 예측한 샘플 비율을 의미하며, 모델이 얼마나 실제 상황을 재현하는지를 나타냄
* 정밀도와 재현율의 가중조화평균인 F1-score라는 지표는 정확도에 비해 더 효과적인 모델 분석 지표로 알려져 있음
* 직접 계산할 수도 있으나, scikit-learn은 이를 편리하게 계산해주는 함수를 제공

* 다중 클래스 분류 문제에서 정밀도와 재현률을 계산할 때는 클래스간의 지표를 어떻게 합칠지 지정 필요

  * None - 클래스간 지표를 합치지 말고 그대로 출력
  * micro - 정밀도와 재현률이 같음, 이로 인해 f1-score도 정밀도, 재현률과 동일
  * macro - 클래스간 지표를 단순 평균한 값
  * weighted - 클래스간 지표를 가중 평균한 값

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score

precision = precision_score(pred, y_test, average='macro')
recall = recall_score(pred, y_test, average='macro')
f1 = f1_score(pred, y_test, average='macro')
print(precision, recall, f1)

#micro : 값3개 다 똑같이 나옴
#macro : 값3개 각각 다르게 나옴옴

0.7688605980687239 0.7880789968023558 0.7738267333256023


## 그리드 검색을 이용한 파라미터 최적화

* 그리드 검색을 사용하면 분류기에 사용하는 파라미터 최적화 가능
* 그리드 검색을 통해 앞서 구성한 나이브 베이즈 모델의 'alpha' 파라미터를 최적화시키는 예제

* `estimator`: 사용 모델 객체     
* `param_grid`: 사용 객체:지정 파라미터 리스트로 구성된 딕셔너리    
* `scoring`: 최적화하고자 하는 성능 지표   
* `cv`: 교차 검증 분할 개수      

In [ ]:
from sklearn.model_selection import GridSearchCV

GS = GridSearchCV(estimator=NB, param_grid={
    'alpha' : [0.001, 0.01, 0.1, 1.]},
    scoring = 'accuracy',
    cv = 10
)

GS.fit(x_train, y_train)
print(GS.best_score_)
print(GS.best_params_)

0.8754879707313336
{'alpha': 0.001}


In [ ]:
from sklearn.model_selection import GridSearchCV

GS = GridSearchCV(estimator=NB, param_grid={
    'alpha' : [0.001, 0.002, 0.003, 0.0005]},
    scoring = 'accuracy',
    cv = 10
)

GS.fit(x_train, y_train)
print(GS.best_score_)
print(GS.best_params_)

0.8763716494911187
{'alpha': 0.0005}
